# Creating a Weather Bot with Rasa stack

In this workshop, we are going to go through a step-by-step process of chatbot development with Rasa stack. The goal of this workshop is to not only to learn how train NLU and dialogue management models but to also create a simple and fully functioning chatbot. The bot which we are going to develop will be able to answer questions about the weather conditions in specific areas in real time. The simplified architecture of what we are going to create looks as follows:

![alt text](./pictures/architecture.png "Title")

# Libraries and tools which will be used to build this bot:

- Python 3
- Code edditor (IDE or Notepad++)


- rasa_nlu
- rasa_core
- sklearn
- sklearn_crfsuite
- spacy
- npm
- apixu 

(all libraries can be installed using requirements.txt file)


# Step One: Building an NLU model

NLU model will be used to extract necessary bits of information from input messages:
 - classify intents (what the input message is about)
 - extract entities (locations, dates, etc.)

## Getting the training data ready

To train an NLU model we have to provide some training data which should consist of example messages and questions which we would like our chatbot to understand and learn from; intents which those example messages correspond to; character positions of where in a sentence our model should find and extract the entities. To get started, in a folder called *data* there is a file called <span style="color:red">demo_data.json</span> which we are going use and add more examples for training. 

We can add new examples to a json file directly, or we can do it using Rasa NLU trainer which has a very user-friendly UI which makes the generation of new training data a lot easier and more comfortable. To launch it, navigate to a folder called *data* and type the following (make sure you have npm and node installed):

### <span style="color:red">rasa-nlu-trainer</span>

It will open the demo_data.json file in a browser. After adding some new examples and saving them using UI buttons, new examples will be automatically added to demo_data.json file for us to use.

![alt text](./pictures/training_data.png "Training data")

Our training data will consist of three different intents: greet, goodbye, inform, and one entity - location.

## Creating a configuration file

Once we have our training data ready, we can start training the model. First, we need a configuration file. A configuration file is used to set what pipeline is going to be used to parse the messages, provide the directory of where the trained model will be saved, and to tell the system where the training data is located. An example configuration file called <span style="color:red">config_spacy.json</span> (it can be found in the main directory) contains the following settings:


In [7]:
### config_spacy.json

config = {
  "pipeline": "spacy_sklearn",
  "path" : "./models/nlu",
  "data" : "./data/demo_data.json"
}

## Training the model

Once we have a configuration file specified, we can train the model. We are going to train the model by running a train_nlu function in <span style="color:red">nlu_model.py</span> file. The code in this function will load the training data, a configuration file and will train the model which will be stored in a ./models/nlu directory.

In [1]:
### nlu_model.py file

from rasa_nlu.converters import load_data
from rasa_nlu.config import RasaNLUConfig
from rasa_nlu.model import Trainer
from rasa_nlu.model import Metadata, Interpreter

def train_nlu(data, config, model_dir):
    training_data = load_data(data)
    trainer = Trainer(RasaNLUConfig(config))
    trainer.train(training_data)
    model_directory = trainer.persist(model_dir, fixed_model_name = 'weathernlu')
    
if __name__ == '__main__':
    #TODO: train the model and test in on input messages
    train_nlu('./data/demo_data.json', 'config_spacy.json', './models/nlu')

Fitting 2 folds for each of 6 candidates, totalling 12 fits


C:\Users\Juste\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Juste\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Juste\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Juste\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Juste\Anaconda3\lib\site-packages\sklearn\metrics\c

Once the model is trained, we can test it out on various inputs. A function run_nlu in <span style="color:red">nlu_model.py</span> file will load the previously trained nlu model as an interpreter and then we will be able to test the model performance on various text messages. The example below shows the model output for a test message 'I am planning my holiday to Barcelona. What is the weather out there at the moment?'.

In [2]:
### run_nlu.py file

from rasa_nlu.converters import load_data
from rasa_nlu.config import RasaNLUConfig
from rasa_nlu.model import Trainer
from rasa_nlu.model import Metadata, Interpreter


def run_nlu():
    #TODO: load the interpreter and test in on input message
    interpreter = Interpreter.load("./models/nlu/default/weathernlu", RasaNLUConfig('config_spacy.json'))
    print(interpreter.parse(u"I am planning my holiday to Barcelona. What is the weather out there at the moment?"))

    
if __name__ == '__main__':
    #TODO: train the model and test in on input messages
    run_nlu()

{'intent': {'name': 'inform', 'confidence': 0.79831883048819108}, 'entities': [{'start': 28, 'end': 37, 'value': 'Barcelona', 'entity': 'location', 'extractor': 'ner_crf'}], 'intent_ranking': [{'name': 'inform', 'confidence': 0.79831883048819108}, {'name': 'goodbye', 'confidence': 0.10245074910386204}, {'name': 'greet', 'confidence': 0.099230420407946662}], 'text': 'I am planning my holiday to Barcelona. What is the weather out there at the moment?'}


The nlu model returns the intent classification results with a confidence level for each intent included in our training data as well as extracted entities. We can see that even though we provided only a handful of training examples, our model is already performing well on out-of-scope messages. We are going to use this model soon, but before that, we shall proceed with the second part of this tutorial - a dialogue management system.

# Step two: Building a model for dialogue management

We are going to use Rasa Core library to train the dialogue managemet model, which will make predictions on what actions our chatbot should perform at specific states of the conversation. There are going to be four actions which we will want our agent to handle:
 - Reply to a greeting message
 - Reply to a goodbye message
 - Answer what's the weather in a specific location
 - If a location is not specified, ask for a location first and then tell what's the weather

## Creating the domain

At first, we will create the domain for our agent. A domain is like a universe in which our bot is going to operate, therefore it has to contain all bits of information which the agent will have to be aware of in order to work properly. In our domain we are going to provide the following details:
- Slots (bits of information which we would like our bot to keep track of throughout the conversation)
- intents (extracted by NLU model)
- entities (extracted by NLU model)
- templates for simple text responses
- actions

We have an example domain saved in <span style="color:red">weather_domain.yml</span> file which we are going to use for our weather bot.

In [ ]:
### weather_domain.yml

slots:
  location:
    type: text
    

intents:
 - greet
 - goodbye
 - inform


entities:
 - location

templates:
  utter_greet:
    - 'Hello! How can I help?'
  utter_goodbye:
    - 'Talk to you later.'
    - 'Bye bye :('
  utter_ask_location:
    - 'In what location?'

actions:
 - utter_greet
 - utter_goodbye
 - utter_ask_location
 - actions.ActionWeather

Most of the actions are going to have simple text answers, but the action which will return the details about the current weather conditions is going to be slightly different - to get the weather data our bot will have to make an api call which will retrieve the details about the current weather conditions in specific areas. For all this, we are going to create a custom action. The file <span style="color:red">actions.py</span> contains the code for a custom action which will make an api call and will get the weather data for the location retrieved from a currently populated slot. An api call is going to return a dictionary with weather-related information, which we are going to parse and use some of those details in creating the response message, which will be sent when this action is triggered. The only thing which you have to provide in this file is apixu key which you can get by signing up at https://www.apixu.com/. This key will be used to make a user authentication and will allow you to make api calls and get weather data.

In [3]:
### actions.py file

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

from rasa_core.actions.action import Action
from rasa_core.events import SlotSet


class ActionWeather(Action):
    def name(self):
        return 'action_weather'

    def run(self, dispatcher, tracker, domain):
        from apixu.client import ApixuClient, ApixuException
        api_key = '' #Provide your apixu Key here
        client = ApixuClient(api_key)
        
        #TODO: retrieve the slot value and make an api call
        loc = tracker.get_slot('location')
        current = client.getCurrentWeather(q=loc)
        
        country = current['location']['country']
        city = current['location']['name']
        condition = current['current']['condition']['text']
        temperature_c = current['current']['temp_c']
        temperature_f = current['current']['temp_f']
        humidity = current['current']['humidity']
        wind_mph = current['current']['wind_mph']
        
        #TODO: create a response message and dispatch it to the output channel
        response = 'It is currently {} in {} at the moment. The temperature is {} degrees, the humidity is {}% and the wind speed is {} mph.'.format(condition, city, temperature_c, humidity, wind_mph)
        dispatcher.utter_message(response)
        return [SlotSet("location", loc)]

## Creating stories

The last piece of dialogue management model puzzle which is needed before we start training the model is training data. Dialogue model is trained on real conversational data, called stories. A story is an actual conversation which the users would have with a bot, converted into a format which Rasa Core can learn from - user inputs are converted to corresponding intents and entities, while the responses sent by a bot are converted to action names which should be executed at a specific state of the conversation. We have already pre-made file with a few stories to train the model on, the file is called <span style="color:red">stories.md</span> and can be found in **data** folder. Below, there is an example of one of the stories which will be used for training:




In [ ]:
### stories.md file

## Generated Story 8921121480760034253
* _greet
    - utter_greet
* _inform
    - utter_ask_location
* _inform[location=London]
    - slot{"location": "London"}
    - action_weather
* _goodbye
    - utter_goodbye

There are few different ways of how we can add more stories to our training data. One of them is to write new stories directly into our data file and another way is to generate new training data during interactive learning sessions.


## Interactive Learning

Interactive learning allows generating new data and training an agent in real time. To start online training we are going to run the script inside the <span style="color:red">train_init.py</span> file, which will train a simple dialogue management model, and just after that we will run the script called <span style="color:red">online_training.py</span>, which will start an online training session. Online training session looks like an actual conversation with the bot, the only difference is that before replying our chatbot will tell us what it thinks it should do at a specific state of the conversation and will ask for our feedback on whether it is about to call the right action or not. Our feedback will be used to train the model on the fly. When you launch an online training session, you should see the following on your console:

![alt text](./pictures/online_training.png "Interactive learning")

With every message we type, we can see how our agent is performing intent classification, entity extraction and what actions it thinks it should make at that state of the conversation. By typing the corresponding number codes, based on whether or not the bot picked the correct actions, we can correct any potential mistakes our bot is about to make. The model improvements are saved automatically, and generated stories can be saved by selecting 'Export current conversations as stories and quit' option on the menu.

## Supervised Learning

Another way of training a dialogue model is called supervised learning - we have training data, we initialise the model with our preferred parameters, and finally we fit the model. The file called <span style="color:red">dialogue_management_model.py</span> contains a function called train_dialogue which we are going to use to train the model. The function will load the training data and policy and will train the model with specified parameters. The model will be saved in a specified location ./models/dialogue.

In [4]:
### train_agent.py file

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import argparse
import logging

from builtins import str


from rasa_core.agent import Agent
from rasa_core.channels.console import ConsoleInputChannel
from rasa_core.interpreter import RasaNLUInterpreter, RegexInterpreter
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy
from rasa_core.interpreter import RegexInterpreter	
from rasa_core.interpreter import RasaNLUInterpreter

logger = logging.getLogger(__name__)

class WeatherPolicy(KerasPolicy):
    def model_architecture(self, num_features, num_actions, max_history_len):
        """Build a Keras model and return a compiled model."""
        from keras.layers import LSTM, Activation, Masking, Dense
        from keras.models import Sequential

        n_hidden = 32  # size of hidden layer in LSTM
        # Build Model
        batch_shape = (None, max_history_len, num_features)

        model = Sequential()
        model.add(Masking(-1, batch_input_shape=batch_shape))
        model.add(LSTM(n_hidden, batch_input_shape=batch_shape))
        model.add(Dense(input_dim=n_hidden, output_dim=num_actions))
        model.add(Activation('softmax'))

        model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

        logger.debug(model.summary())
        return model
        

def train_dialogue(domain_file='weather_domain.yml',
                   model_path='./models/dialogue',
                   training_data_file='./data/stories.md'):
    agent = Agent(domain_file,
                  policies=[MemoizationPolicy(), WeatherPolicy()])

    agent.train(#TODO set the model parameters
                training_data_file,
                max_history = 3,
                epochs = 300,
                batch_size = 50,
                augmentation_factor = 50,
                validation_split = 0.2
    )

    agent.persist(model_path)
    return agent




if __name__ == '__main__':
    #TODO: train the model and test it
    train_dialogue()


Using TensorFlow backend.
C:\Users\Juste\Anaconda3\lib\site-packages\pykwalify\core.py:76: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  self.source = yaml.load(stream)
Processed Story Blocks: 100%|█████████████████████████████████| 9/9 [00:00<00:00, 80.17it/s, # trackers=9, samples=273]
C:\Users\Juste\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=32, units=7)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, 3, 12)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                5760      
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 231       
_________________________________________________________________
activation_1 (Activation)    (None, 7)                 0         
Total params: 5,991
Trainable params: 5,991
Non-trainable params: 0
_________________________________________________________________
Train on 17 samples, validate on 5 samples
Epoch 1/300
17/17 [==============================] - 0s - loss: 2.0007 - acc: 0.1765 - val_loss: 1.9471 - val_acc: 0.2000
Epoch 2/300
17/17 [==============================] - 0s - loss: 1.9909 - acc: 0.1765 - val_loss: 1.9395 - v

17/17 [==============================] - 0s - loss: 0.8016 - acc: 0.7059 - val_loss: 1.8006 - val_acc: 0.4000
Epoch 127/300
17/17 [==============================] - 0s - loss: 0.7969 - acc: 0.7059 - val_loss: 1.8041 - val_acc: 0.4000
Epoch 128/300
17/17 [==============================] - 0s - loss: 0.7921 - acc: 0.7059 - val_loss: 1.8076 - val_acc: 0.4000
Epoch 129/300
17/17 [==============================] - 0s - loss: 0.7873 - acc: 0.7059 - val_loss: 1.8111 - val_acc: 0.4000
Epoch 130/300
17/17 [==============================] - 0s - loss: 0.7824 - acc: 0.7059 - val_loss: 1.8147 - val_acc: 0.4000
Epoch 131/300
17/17 [==============================] - 0s - loss: 0.7775 - acc: 0.7059 - val_loss: 1.8182 - val_acc: 0.4000
Epoch 132/300
17/17 [==============================] - 0s - loss: 0.7726 - acc: 0.7059 - val_loss: 1.8218 - val_acc: 0.4000
Epoch 133/300
17/17 [==============================] - 0s - loss: 0.7677 - acc: 0.7059 - val_loss: 1.8253 - val_acc: 0.4000
Epoch 134/300
17/17 [=

17/17 [==============================] - 0s - loss: 0.2562 - acc: 0.9412 - val_loss: 2.5619 - val_acc: 0.4000
Epoch 259/300
17/17 [==============================] - 0s - loss: 0.2545 - acc: 0.9412 - val_loss: 2.5665 - val_acc: 0.4000
Epoch 260/300
17/17 [==============================] - 0s - loss: 0.2528 - acc: 0.9412 - val_loss: 2.5710 - val_acc: 0.4000
Epoch 261/300
17/17 [==============================] - 0s - loss: 0.2512 - acc: 0.9412 - val_loss: 2.5755 - val_acc: 0.4000
Epoch 262/300
17/17 [==============================] - 0s - loss: 0.2495 - acc: 0.9412 - val_loss: 2.5799 - val_acc: 0.4000
Epoch 263/300
17/17 [==============================] - 0s - loss: 0.2479 - acc: 0.9412 - val_loss: 2.5844 - val_acc: 0.4000
Epoch 264/300
17/17 [==============================] - 0s - loss: 0.2463 - acc: 0.9412 - val_loss: 2.5888 - val_acc: 0.4000
Epoch 265/300
17/17 [==============================] - 0s - loss: 0.2448 - acc: 0.9412 - val_loss: 2.5931 - val_acc: 0.4000
Epoch 266/300
17/17 [=

# Step 3: Testing the Agent

Once we train the model,  it is time to test it out. For this workshop, we are going to use our terminal as a communication channel. The file called <span style="color:red">dialogue_management_model.py</span> has a function run_weather_bot which will launch the agent in our command line. The code in this file will load the NLU interpreter which will parse incoming messages, and load the dialogue management model which will make predictions on what action our agent should make. To launch the bot open the terminal and run the dialogue_management_scirpt.py script and call run_weather_bot function.

In [ ]:
def run_weather_bot(serve_forever=True):
    #TODO: Load an interpreter and the agent
    interpreter = RasaNLUInterpreter('./models/nlu/default/weathernlu')
    agent = Agent.load('./models/dialogue', interpreter = interpreter)
    
    #TODO: start listening to incoming messages
    if serve_forever:
        agent.handle_channel(ConsoleInputChannel())
    return agent

if __name__ == '__main__':
    #TODO: train the model and test it
    run_weather_bot()

And that's it - we created a simple chatbot which can report weather condition in real time. Once loaded, you can have a conversation with your bot and test its performance. Keep in mind, that we used a very small training data samples for both NLU and Dialogue Management models, but you should still be able to get a fairly good performance. One of the conversations with your chatbot may look like this:

![alt text](./pictures/conversation.png "Conversation")

# Step 4: Improve the weather bot/ build something awsome!

Some resources you may find useful:

- https://rasahq.github.io/rasa_nlu/index.html (the official documentation of Rasa NLU)
- https://core.rasa.ai/ (the official documentaion of Rasa Core)
- https://arxiv.org/abs/1712.05181 (a paper on Rasa Stack)
- https://github.com/apixu/apixu-python (the documentation of weather API used in this tutorial)